In [89]:
import requests
import csv
import bitmex
import copy
import numpy as np
import itertools
client=bitmex.bitmex(api_key='eYt_xXuHDVwoyCzDllIGbv3A', api_secret='pUv0ZUt_ri8rcfAwx80BvAGH68Zx-ThmMQ2huMopdZtrjvdH')
print("Starting data gen...")

coin=[]
#BUILD A DATASET USING BITMEX API CALLS FOR ORDERBOOK AND MARKET STATS
#TO GET MORE DATA, CHANGE # Samples
samples=100
#TO GET DATA FROM DIFFERENT COIN, CHANGE VALUES NEXT TO SYMBOL AND CURRENCY BELOW

for i in range(samples):
    print("Iteration: "+str(i+1))
    temp_coin=[]
    print("Polling API for data...")
    result=client.OrderBook.OrderBook_getL2(symbol="XBTUSD").result()
    result2=client.Stats.Stats_history().result()
    for i in range(len(result)-1):
        for j in range(len(result[0])-1):
            for k in range(len(result2)-1):
                for l in range(len(result2[0])-1):
                    if result[i][j]["symbol"]=='XBTUSD' and (result2[k][l]["currency"] == 'XBT' or result2[k][l]["currency"] == 'XBt'):
                        temp_coin.append((result[i][j],result2[k][l]))

                        
    #MERGE 2 DICTS FROM 2 API CALLS INTO SINGLE DICT                        
    temp=0
    merge=[]
    print("Merging dicts...")
    for i in range(len(temp_coin)):
        t=temp_coin[i][0]
        t.update(temp_coin[i][1])
        merge.append(t)            
    temp_coin=merge            

    #TRIM DUPLICATE ENTRIES
    temp_temp_coin=[]
    id_list=[]
    print("Trimming duplicates...")
    for j in range(len(temp_coin)):
        if temp_coin[j]['id'] not in id_list:
            id_list.append(temp_coin[j]['id'])
            temp_temp_coin.append(temp_coin[j])
    temp_coin=temp_temp_coin

    #LABEL TRAINING DATA BASED ON PROFITABILITY OF TRADE
    print("Labeling Training Data...")
    for i in range(len(temp_coin)):
        temp_coin[i]['temp_label']=i
    groups = itertools.groupby(temp_coin, lambda i: i['side'])
    buy_sell_breakdown=[]
    for key, group in groups:
        buy_sell_breakdown.append(list(group))
    count=0
    del buy_sell_breakdown[0][-1]
    for i in range(len(buy_sell_breakdown[0])-1):
        sale_margin = buy_sell_breakdown[0][i]['price']-buy_sell_breakdown[1][i]['price']
        trade_margin = buy_sell_breakdown[0][i]['price']*sale_margin*.05 - buy_sell_breakdown[1][i]['price']*sale_margin*.05
        buy_value_shift = buy_sell_breakdown[0][i]['price'] - buy_sell_breakdown[1][i+1]['price'] 
        sell_value_shift = buy_sell_breakdown[0][i+1]['price']-buy_sell_breakdown[1][i]['price']
        moe=.0003*buy_sell_breakdown[0][i]['price']
        if trade_margin > 0 and (buy_value_shift - sell_value_shift > moe):
            label_to_add=1
        else:
            label_to_add=-1
        search_labels=[buy_sell_breakdown[0][i]['temp_label'],buy_sell_breakdown[1][i]['temp_label']]
        for j in range(len(temp_coin)):
            if temp_coin[j]['temp_label'] in search_labels:
                temp_coin[j]['label']=label_to_add
        
    #TRIM UNIMPORTANT FEATURES
    print("Trimming bad features...")
    for k in range(len(temp_coin)):
        del temp_coin[k]['id']
        del temp_coin[k]['rootSymbol']
        del temp_coin[k]['currency']
        del temp_coin[k]['symbol']
        del temp_coin[k]['date']
        del temp_coin[k]['temp_label']
    coin.append(temp_coin)
    

#CONCATENATE SAMPLES    
t_diddy=[]
print("Concatenating samples...")
for i in range(len(coin)):
    t_diddy=np.concatenate((t_diddy,coin[i]))
coin=t_diddy
coin=list(coin)
slated_for_removal=[]
for j in range(len(coin)):
    if 'label' not in coin[j].keys():
        slated_for_removal.append(j)
coin = [i for n, i in enumerate(coin) if n not in slated_for_removal]

print("Produced training data with " +str(len(coin)) +" data points")
#WRITE TO CSV
print("Writing to csv...")
csv_columns = list(coin[0].keys())
csv_file = "coin_train_data.csv"
try:
    with open(csv_file, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
        for data in coin:
            writer.writerow(data)
except IOError:
    print("I/O error")
print("Done")


Starting data gen...
Iteration: 1
Polling API for data...
Merging dicts...
Trimming duplicates...
Labeling Training Data...
Trimming bad features...
Iteration: 2
Polling API for data...
Merging dicts...
Trimming duplicates...
Labeling Training Data...
Trimming bad features...
Iteration: 3
Polling API for data...
Merging dicts...
Trimming duplicates...
Labeling Training Data...
Trimming bad features...
Iteration: 4
Polling API for data...
Merging dicts...
Trimming duplicates...
Labeling Training Data...
Trimming bad features...
Iteration: 5
Polling API for data...
Merging dicts...
Trimming duplicates...
Labeling Training Data...
Trimming bad features...
Iteration: 6
Polling API for data...
Merging dicts...
Trimming duplicates...
Labeling Training Data...
Trimming bad features...
Iteration: 7
Polling API for data...
Merging dicts...
Trimming duplicates...
Labeling Training Data...
Trimming bad features...
Iteration: 8
Polling API for data...
Merging dicts...
Trimming duplicates...
Labeli

Trimming duplicates...
Labeling Training Data...
Trimming bad features...
Iteration: 65
Polling API for data...
Merging dicts...
Trimming duplicates...
Labeling Training Data...
Trimming bad features...
Iteration: 66
Polling API for data...
Merging dicts...
Trimming duplicates...
Labeling Training Data...
Trimming bad features...
Iteration: 67
Polling API for data...
Merging dicts...
Trimming duplicates...
Labeling Training Data...
Trimming bad features...
Iteration: 68
Polling API for data...
Merging dicts...
Trimming duplicates...
Labeling Training Data...
Trimming bad features...
Iteration: 69
Polling API for data...
Merging dicts...
Trimming duplicates...
Labeling Training Data...
Trimming bad features...
Iteration: 70
Polling API for data...
Merging dicts...
Trimming duplicates...
Labeling Training Data...
Trimming bad features...
Iteration: 71
Polling API for data...
Merging dicts...
Trimming duplicates...
Labeling Training Data...
Trimming bad features...
Iteration: 72
Polling A